# CRUCE CON INTENSIDAD DEL TRAFICO

In [1]:
import pandas as pd
import numpy as np

In [2]:
path_traff_may_2019 = '/home/dsc/Master DS/TFM/TFM INFO/traffic/2019/05-2019.csv'

In [3]:
traff_may_2019 = pd.read_csv(path_traff_may_2019, sep = ';' )

In [4]:
traff_may_2019.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
0,1001,2019-05-01 00:00:00,M30,732,2.0,0,60.0,N,5
1,1001,2019-05-01 00:15:00,M30,804,3.0,0,63.0,N,5
2,1001,2019-05-01 00:30:00,M30,516,1.0,0,60.0,N,5
3,1001,2019-05-01 00:45:00,M30,780,2.0,0,58.0,N,5
4,1001,2019-05-01 01:00:00,M30,780,2.0,0,59.0,N,5


Del dataset eliminamos las siguientes partes:
- periodo_integracion
- vmed 
- tipo_elem: las mediciones referidas a identificativos de medición en la M30

In [5]:
traff_may_2019_clean = traff_may_2019.drop(['periodo_integracion','vmed'], axis=1)


In [6]:
cond_M30 = traff_may_2019_clean[traff_may_2019_clean['tipo_elem']=='M30'].index
traff_may_2019_clean.drop(cond_M30,inplace=True)

In [52]:
print(traff_may_2019.shape)
print(traff_may_2019_clean.shape)

(11060255, 9)
(9837607, 7)


Vamos a comprobar la existencia de Nans en el dataset

In [7]:
traff_may_2019_columns = traff_may_2019_clean.columns.str.strip()
traff_may_2019_columns

Index(['id', 'fecha', 'tipo_elem', 'intensidad', 'ocupacion', 'carga',
       'error'],
      dtype='object')

In [11]:
traff_may_2019_df=pd.DataFrame(traff_may_2019_clean,columns=traff_may_2019_columns )

In [12]:
traff_may_2019_Nan = traff_may_2019_df[traff_may_2019_columns].isna().sum().reset_index()
traff_may_2019_Nan


,index,0
0,id,0
1,fecha,0
2,tipo_elem,0
3,intensidad,0
4,ocupacion,0
5,carga,0
6,error,0


El tráfico lo vamos a categorizar entre los siguientes estados:
- tráfico fluido (nivel 0)
- tráfico lento (nivel 1)
- retenciones (nivel 2)
- congestión (nivel 3)

La “carga” es un dato calculado a partir de una fórmula polinómica que tiene en cuenta datos de la intensidad real, ocupación, intensidad de saturación y otros parámetros. En cuanto al término “intensidad de saturación”, es el número máximo de vehículos que pueden pasar en una hora por un punto de la vía, teniendo en cuenta todos los carriles de circulación de la sección de la vía

Hacemos con la fecha las siguientes columnas:
- hora
- día de la semana
- fecha

In [18]:
aaa=traff_may_2019_df.head(50)

In [21]:
separado=aaa["fecha"].str.split(" ", n=1, expand=True)

In [22]:
aaa["date"]= separado[0]

/tmp/ipykernel_3672/2344899064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aaa["date"]= separado[0]


In [24]:
aaa['hour']=separado[1]

/tmp/ipykernel_3672/3372768314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aaa['hour']=separado[1]


In [26]:
aaa.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,error,date,hour
139872,3395,2019-05-01 00:00:00,URB,207,2.0,13,N,2019-05-01,00:00:00
139873,3395,2019-05-01 00:15:00,URB,220,2.0,14,N,2019-05-01,00:15:00
139874,3395,2019-05-01 00:30:00,URB,196,2.0,13,N,2019-05-01,00:30:00
139875,3395,2019-05-01 00:45:00,URB,169,2.0,12,N,2019-05-01,00:45:00
139876,3395,2019-05-01 01:00:00,URB,176,4.0,13,N,2019-05-01,01:00:00


In [37]:
import datetime 
import calendar

In [46]:
def findDay(date): 
    born = datetime.datetime.strptime(date, '%Y-%m-%d').weekday() 
    return (calendar.day_name[born]) 

In [47]:
findDay('2019-05-01')

'Wednesday'

In [48]:
aaa['weak_day']=aaa['date'].apply(findDay) 

/tmp/ipykernel_3672/3834324745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aaa['weak_day']=aaa['date'].apply(findDay)


In [49]:
aaa.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,error,date,hour,weak_day
139872,3395,2019-05-01 00:00:00,URB,207,2.0,13,N,2019-05-01,00:00:00,Wednesday
139873,3395,2019-05-01 00:15:00,URB,220,2.0,14,N,2019-05-01,00:15:00,Wednesday
139874,3395,2019-05-01 00:30:00,URB,196,2.0,13,N,2019-05-01,00:30:00,Wednesday
139875,3395,2019-05-01 00:45:00,URB,169,2.0,12,N,2019-05-01,00:45:00,Wednesday
139876,3395,2019-05-01 01:00:00,URB,176,4.0,13,N,2019-05-01,01:00:00,Wednesday


In [29]:
num_id=traff_may_2019_clean_df.groupby('id').count()
num_id

,fecha,tipo_elem,intensidad,ocupacion,carga,error
id,,,,,,
3395,2878,2878,2878,2878,2878,2878
3396,2877,2877,2877,2877,2877,2877
3397,2898,2898,2898,2898,2898,2898
3398,2784,2784,2784,2784,2784,2784
3399,2578,2578,2578,2578,2578,2578
...,...,...,...,...,...,...
10409,2880,2880,2880,2880,2880,2880
10410,1102,1102,1102,1102,1102,1102
10412,45,45,45,45,45,45


Criterios de filtrado:
- tipo_elem: URB